In [1]:
# 📦 1. Instalar dependencias
!pip install -q transformers datasets peft accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 79.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 62.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 28.2 MB/s eta 0:00:00


In [2]:
from google.colab import drive  # Importing the library to mount Google Drive
drive.mount('/content/drive')  # Mounting Google Drive in Colab environment

Mounted at /content/drive


In [5]:
import pandas as pd

# File paths
train_df_file = "/content/drive/My Drive/MOE_DGA/train_wl.csv"

train_df = pd.read_csv(train_df_file)
train_df["Labels"]=train_df["label"]
print(train_df)

                       domain    family   label  Labels
0         nailconsiderable.ru  suppobox     dga     dga
1            stilldelight.net  suppobox     dga     dga
2       kimberleekatheryn.net  suppobox     dga     dga
3                soilbeen.net  suppobox     dga     dga
4               visitform.net  suppobox     dga     dga
...                       ...       ...     ...     ...
159995             dhuhaa.com     legit  notdga  notdga
159996        sdmetalcrew.org     legit  notdga  notdga
159997  melbcampcontuligol.ga     legit  notdga  notdga
159998      pl-enthusiast.net     legit  notdga  notdga
159999            rd-forum.ru     legit  notdga  notdga

[160000 rows x 4 columns]


In [6]:


# 📚 2. Importar librerías necesarias
import torch
import pandas as pd
from datasets import Dataset
from transformers import BertTokenizer, BertForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding
from peft import LoraConfig, get_peft_model
import os
os.environ["WANDB_DISABLED"] = "true"

# ⚙️ 3. Detectar dispositivo (GPU si está disponible)
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"🔹 Usando dispositivo: {device.upper()}")

# 📑 4. Suponemos que ya tienes train_df cargado con columnas: domain y Label
# Por ejemplo, podrías cargarlo desde un CSV:
# train_df = pd.read_csv("tus_datos.csv")

# 🔤 5. Preprocesamiento
label_map = {"notdga": 0, "dga": 1}
train_df["label"] = train_df["Labels"].map(label_map)
dataset = Dataset.from_pandas(train_df[["domain", "label"]])

# 🧠 6. Tokenizador y tokenización
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def tokenize_fn(batch):
    return tokenizer(batch["domain"], truncation=True)

tokenized_dataset = dataset.map(tokenize_fn, batched=True)


🔹 Usando dispositivo: CUDA


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Map:   0%|          | 0/160000 [00:00<?, ? examples/s]

In [8]:

# 🏗️ 7. Cargar modelo base y aplicar LoRA
base_model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["query", "value"],  # capas internas válidas para LoRA
    lora_dropout=0.1,
    bias="none",
    task_type="SEQ_CLS"
)

model = get_peft_model(base_model, lora_config)
model.to(device)

# 🛠️ 8. Configuración de entrenamiento
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="no",
    per_device_train_batch_size=256,
    num_train_epochs=3,
    logging_steps=1000,
    save_strategy="no",
    learning_rate=2e-5,
    weight_decay=0.01
)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# 🏋️ 9. Entrenador
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator
)

# 🚀 10. Entrenamiento
trainer.train()

# 💾 11. Guardar modelo
model.save_pretrained("./dga_detector_lora")
tokenizer.save_pretrained("./dga_detector_lora")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-8-fedf4dd7e64e>:31: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
1000,0.529600


('./dga_detector_lora/tokenizer_config.json',
 './dga_detector_lora/special_tokens_map.json',
 './dga_detector_lora/vocab.txt',
 './dga_detector_lora/added_tokens.json')

In [9]:
from transformers import BertTokenizer, BertForSequenceClassification
from peft import PeftModel
import torch

# ✅ Cargar modelo y tokenizer
model_path = "./dga_detector_lora"  # Ruta donde guardaste el modelo
tokenizer = BertTokenizer.from_pretrained(model_path)

base_model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
model = PeftModel.from_pretrained(base_model, model_path)
model.eval()

# ✅ Detectar dispositivo
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# ✅ Diccionario inverso de etiquetas
id2label = {0: "notdga", 1: "dga"}

# ✅ Función de predicción
def predict_domain(domain_name: str):
    inputs = tokenizer(domain_name, return_tensors="pt", truncation=True).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
        predicted_class = torch.argmax(outputs.logits, dim=1).item()
        return id2label[predicted_class]

# 🔍 Ejemplo
dominio = "google.com"
prediccion = predict_domain(dominio)
print(f"🔎 Dominio: {dominio} → Predicción: {prediccion}")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


🔎 Dominio: google.com → Predicción: notdga


In [12]:
import os
import time
import torch
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
from transformers import BertTokenizer, BertForSequenceClassification
from peft import PeftModel

# ✅ Cargar modelo y tokenizer (ajustar ruta si es necesario)
model_path = "./dga_detector_lora"
tokenizer = BertTokenizer.from_pretrained(model_path)
base_model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
model = PeftModel.from_pretrained(base_model, model_path).eval().to("cuda" if torch.cuda.is_available() else "cpu")

# ✅ Diccionario inverso
id2label = {0: "notdga", 1: "dga"}

# ✅ Función de predicción
def predict(domain):
    inputs = tokenizer(domain, return_tensors="pt", truncation=True).to(model.device)
    with torch.no_grad():
        outputs = model(**inputs)
        predicted_class = torch.argmax(outputs.logits, dim=1).item()
    return predicted_class

# ✅ Familias que quieres evaluar
families = [
    'matsnu.gz',
    'suppobox.gz',
    'charbot.gz',
    'gozi.gz',
    'manuelita.gz',
    'rovnix.gz',
    'deception.gz',
    'nymaim.gz'
]

runs = 30
output_dir = "/content/drive/My Drive/results"
os.makedirs(output_dir, exist_ok=True)

# ✅ Evaluación por familia
for family in families:
    print(f"Processing family: {family}")

    # ⚠️ Asume que estos archivos tienen columnas: domain, label
    dga = pd.read_csv(f'/content/drive/My Drive/Familias_Test/{family}', chunksize=50)
    legit = pd.read_csv('/content/drive/My Drive/Familias_Test/legit.gz', chunksize=50)

    for run in range(runs):
        print(f'Run {run + 1}/{runs} - {family}', end='\r')

        # Obtener un chunk de cada uno
        df_dga = dga.get_chunk()
        df_legit = legit.get_chunk()

        # Agregar columna label
        df_dga["label"] = 1
        df_legit["label"] = 0

        # Mezclar y preparar
        dfw = pd.concat([df_dga, df_legit]).sample(frac=1).reset_index(drop=True)

        pred = []
        query_time = []

        for domain in dfw["domain"]:
            start = time.time()
            label = predict(domain)
            pred.append(label)
            query_time.append(time.time() - start)

        dfw["pred"] = pred
        dfw["query_time"] = query_time

        # ✅ Métricas (opcional)
        acc = accuracy_score(dfw["label"], dfw["pred"])
        f1 = f1_score(dfw["label"], dfw["pred"])
        print(f"Run {run+1} | ACC: {acc:.4f} | F1: {f1:.4f}")

        # ✅ Guardar resultados
        output_file = os.path.join(output_dir, f'results_LORA_classifier_{family}_{run}.csv.gz')
        dfw.to_csv(output_file, index=False)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Processing family: matsnu.gz
Run 1 | ACC: 0.8800 | F1: 0.8776
Run 2 | ACC: 0.8900 | F1: 0.8911
Run 3 | ACC: 0.9000 | F1: 0.9020
Run 4 | ACC: 0.8000 | F1: 0.8113
Run 5 | ACC: 0.8600 | F1: 0.8679
Run 6 | ACC: 0.9300 | F1: 0.9293
Run 7 | ACC: 0.8700 | F1: 0.8762
Run 8 | ACC: 0.9000 | F1: 0.8936
Run 9 | ACC: 0.8600 | F1: 0.8571
Run 10 | ACC: 0.8800 | F1: 0.8824
Run 11 | ACC: 0.9200 | F1: 0.9184
Run 12 | ACC: 0.9600 | F1: 0.9608
Run 13 | ACC: 0.8900 | F1: 0.8952
Run 14 | ACC: 0.8600 | F1: 0.8679
Run 15 | ACC: 0.8700 | F1: 0.8713
Run 16 | ACC: 0.8900 | F1: 0.8932
Run 17 | ACC: 0.9200 | F1: 0.9245
Run 18 | ACC: 0.9300 | F1: 0.9320
Run 19 | ACC: 0.8800 | F1: 0.8846
Run 20 | ACC: 0.9000 | F1: 0.9020
Run 21 | ACC: 0.8800 | F1: 0.8800
Run 22 | ACC: 0.9000 | F1: 0.9020
Run 23 | ACC: 0.8700 | F1: 0.8762
Run 24 | ACC: 0.8400 | F1: 0.8431
Run 25 | ACC: 0.8500 | F1: 0.8544
Run 26 | ACC: 0.9400 | F1: 0.9400
Run 27 | ACC: 0.9200 | F1: 0.9216
Run 28 | ACC: 0.9200 | F1: 0.9200
Run 29 | ACC: 0.9300 | F1: 0

In [14]:
import os
import time
import torch
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
from transformers import BertTokenizer, BertForSequenceClassification
from peft import PeftModel

# ✅ Cargar modelo y tokenizer (ajustar ruta si es necesario)
model_path = "./dga_detector_lora"
tokenizer = BertTokenizer.from_pretrained(model_path)
base_model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
model = PeftModel.from_pretrained(base_model, model_path).eval().to("cuda" if torch.cuda.is_available() else "cpu")

# ✅ Diccionario inverso
id2label = {0: "notdga", 1: "dga"}

# ✅ Función de predicción
def predict(domain):
    inputs = tokenizer(domain, return_tensors="pt", truncation=True).to(model.device)
    with torch.no_grad():
        outputs = model(**inputs)
        predicted_class = torch.argmax(outputs.logits, dim=1).item()
    return predicted_class

# ✅ Familias que quieres evaluar
families = ['bigviktor.gz',
            'pizd.gz',
            'ngioweb.gz'


           ]

runs = 30
output_dir = "/content/drive/My Drive/results"
os.makedirs(output_dir, exist_ok=True)

# ✅ Evaluación por familia
for family in families:
    print(f"Processing family: {family}")

    # ⚠️ Asume que estos archivos tienen columnas: domain, label
    dga = pd.read_csv(f'/content/drive/My Drive/New_Families/{family}', chunksize=50)
    legit = pd.read_csv('/content/drive/My Drive/Familias_Test/legit.gz', chunksize=50)

    # Saltar los primeros 30 chunks de legit
    for _ in range(30):
        legit.get_chunk()


    for run in range(runs):
        print(f'Run {run + 1}/{runs} - {family}', end='\r')

        # Obtener un chunk de cada uno
        df_dga = dga.get_chunk()
        df_legit = legit.get_chunk()

        # Agregar columna label
        df_dga["label"] = 1
        df_legit["label"] = 0

        # Mezclar y preparar
        dfw = pd.concat([df_dga, df_legit]).sample(frac=1).reset_index(drop=True)

        pred = []
        query_time = []

        for domain in dfw["domain"]:
            start = time.time()
            label = predict(domain)
            pred.append(label)
            query_time.append(time.time() - start)

        dfw["pred"] = pred
        dfw["query_time"] = query_time

        # ✅ Métricas (opcional)
        acc = accuracy_score(dfw["label"], dfw["pred"])
        f1 = f1_score(dfw["label"], dfw["pred"])
        print(f"Run {run+1} | ACC: {acc:.4f} | F1: {f1:.4f}")

        # ✅ Guardar resultados
        output_file = os.path.join(output_dir, f'results_LORA_classifier_{family}_{run}.csv.gz')
        dfw.to_csv(output_file, index=False)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Processing family: bigviktor.gz
Run 1 | ACC: 0.8400 | F1: 0.8182
Run 2 | ACC: 0.7400 | F1: 0.7111
Run 3 | ACC: 0.8500 | F1: 0.8352
Run 4 | ACC: 0.7700 | F1: 0.7527
Run 5 | ACC: 0.7600 | F1: 0.7447
Run 6 | ACC: 0.7900 | F1: 0.7789
Run 7 | ACC: 0.8200 | F1: 0.8125
Run 8 | ACC: 0.8300 | F1: 0.8172
Run 9 | ACC: 0.7800 | F1: 0.7500
Run 10 | ACC: 0.8200 | F1: 0.8163
Run 11 | ACC: 0.8400 | F1: 0.8261
Run 12 | ACC: 0.7400 | F1: 0.7174
Run 13 | ACC: 0.7900 | F1: 0.7640
Run 14 | ACC: 0.8400 | F1: 0.8182
Run 15 | ACC: 0.8000 | F1: 0.7826
Run 16 | ACC: 0.7900 | F1: 0.7640
Run 17 | ACC: 0.8000 | F1: 0.7872
Run 18 | ACC: 0.8900 | F1: 0.8817
Run 19 | ACC: 0.8200 | F1: 0.7955
Run 20 | ACC: 0.8400 | F1: 0.8400
Run 21 | ACC: 0.8200 | F1: 0.8043
Run 22 | ACC: 0.7400 | F1: 0.7174
Run 23 | ACC: 0.8500 | F1: 0.8454
Run 24 | ACC: 0.8300 | F1: 0.8172
Run 25 | ACC: 0.7900 | F1: 0.7742
Run 26 | ACC: 0.8400 | F1: 0.8222
Run 27 | ACC: 0.7600 | F1: 0.7273
Run 28 | ACC: 0.7900 | F1: 0.7586
Run 29 | ACC: 0.8300 | F1

In [15]:
import os
import numpy as np
import pandas as pd
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, confusion_matrix
)
import matplotlib.pyplot as plt
import seaborn as sns

# Montar Google Drive (si aún no está montado)
from google.colab import drive
drive.mount('/content/drive')

# Función para calcular FPR y TPR
def fpr_tpr(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    fpr = fp / (fp + tn + 1e-10)  # Evita división por cero
    tpr = tp / (tp + fn + 1e-10)
    return fpr, tpr

# Lista de familias de malware
families = [
    'matsnu.gz',
    'suppobox.gz',
    'charbot.gz',
    'gozi.gz',
    'manuelita.gz',
    'rovnix.gz',
    'deception.gz',
    'nymaim.gz',
    'bigviktor.gz',
    'pizd.gz',
    'ngioweb.gz'
]
runs = 30

# Listas para métricas globales
all_acc, all_pre, all_rec, all_f1 = [], [], [], []
all_fpr, all_tpr, all_qt, all_qts = [], [], [], []
total_unknowns_global = 0

for family in families:
    acc, pre, rec, f1, fpr, tpr, qt, qts = [], [], [], [], [], [], [], []
    total_unknowns = 0

    for run in range(runs):
        path = f'/content/drive/My Drive/results/results_LORA_classifier_{family}_{run}.csv.gz'
        if not os.path.exists(path):
            print(f"[⚠️] Archivo no encontrado: {path}")
            continue

        df = pd.read_csv(path)

        # Reemplazar -1 por 1 en 'pred', si aplica

        y_true = df['label']
        y_pred = df['pred']

        # Métricas
        acc.append(accuracy_score(y_true, y_pred))
        pre.append(precision_score(y_true, y_pred, zero_division=0))
        rec.append(recall_score(y_true, y_pred, zero_division=0))
        f1.append(f1_score(y_true, y_pred, zero_division=0))
        fpr_val, tpr_val = fpr_tpr(y_true, y_pred)
        fpr.append(fpr_val)
        tpr.append(tpr_val)

        if 'query_time' in df.columns:
            qt.append(df['query_time'].mean())
            qts.append(df['query_time'].std())

    # Promedios por familia
    if acc:  # solo si hubo archivos válidos
        print(f'{family.split(".")[0]:15}: '
              f'acc:{np.mean(acc):.2f}±{np.std(acc):.3f} '
              f'f1:{np.mean(f1):.2f}±{np.std(f1):.3f} '
              f'pre:{np.mean(pre):.2f}±{np.std(pre):.3f} '
              f'rec:{np.mean(rec):.2f}±{np.std(rec):.3f} '
              f'FPR:{np.mean(fpr):.2f}±{np.std(fpr):.3f} '
              f'TPR:{np.mean(tpr):.2f}±{np.std(tpr):.3f} '
              f'QT:{np.mean(qt):.5f}±{np.std(qt):.5f} '
              f'Unknowns: {total_unknowns}')

        all_acc.append(np.mean(acc))
        all_pre.append(np.mean(pre))
        all_rec.append(np.mean(rec))
        all_f1.append(np.mean(f1))
        all_fpr.append(np.mean(fpr))
        all_tpr.append(np.mean(tpr))
        all_qt.append(np.mean(qt))
        all_qts.append(np.mean(qts))
        total_unknowns_global += total_unknowns

# 🔍 Métricas globales
print("\n### 📊 Métricas globales ###")
print(f'Accuracy   : {np.mean(all_acc):.2f}')
print(f'F1-Score   : {np.mean(all_f1):.2f}')
print(f'Precision  : {np.mean(all_pre):.2f}')
print(f'Recall     : {np.mean(all_rec):.2f}')
print(f'FPR        : {np.mean(all_fpr):.2f}')
print(f'TPR        : {np.mean(all_tpr):.2f}')
print(f'Query time : {np.mean(all_qt):.5f} ± {np.mean(all_qts):.5f}')
print(f'Total unknown classifications: {total_unknowns_global}')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
matsnu         : acc:0.89±0.035 f1:0.90±0.033 pre:0.88±0.043 rec:0.91±0.039 FPR:0.13±0.050 TPR:0.91±0.039 QT:0.01345±0.00183 Unknowns: 0
suppobox       : acc:0.88±0.050 f1:0.88±0.059 pre:0.88±0.041 rec:0.89±0.099 FPR:0.13±0.050 TPR:0.89±0.099 QT:0.01341±0.00184 Unknowns: 0
charbot        : acc:0.71±0.041 f1:0.66±0.053 pre:0.82±0.061 rec:0.56±0.061 FPR:0.13±0.050 TPR:0.56±0.061 QT:0.01333±0.00165 Unknowns: 0
gozi           : acc:0.71±0.086 f1:0.64±0.121 pre:0.81±0.066 rec:0.54±0.176 FPR:0.13±0.050 TPR:0.54±0.176 QT:0.01340±0.00165 Unknowns: 0
manuelita      : acc:0.52±0.043 f1:0.24±0.084 pre:0.55±0.157 rec:0.16±0.058 FPR:0.13±0.050 TPR:0.16±0.058 QT:0.01345±0.00167 Unknowns: 0
rovnix         : acc:0.92±0.024 f1:0.92±0.022 pre:0.89±0.039 rec:0.97±0.026 FPR:0.13±0.050 TPR:0.97±0.026 QT:0.01354±0.00168 Unknowns: 0
deception      : acc:0.92±0.030 f1:0.93±0.027 pre